# W261 Final Project: Click Through Rate Prediction
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

__`Rutika Banakar, Stanley Opara, Subha Vadakkumkkor`__


# Section 1: Introduction

Online advertising is a major business for internet companies and one of the core problem in that field is to be able to match the right advertisement to the right user at the right time. For ads on most websites and search engines, advertisers only pay for measurable user responses such are clicks on the ads and other actions that users take on the website such as registering, purchasing etc. Thus an optimal approach (as a search engine) is to choose an ad based on expected value, which is price of a click times the likelihood that the ad will be clicked. In order for websites to maximize expected value, they need to accurately predict the likelihood that a given ad will be clicked, also known as click-through rate (CTR).

# Notebook Set-Up


In [1]:
# imports
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "hw5_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

# Section 2: Problem definition and Dataset

Given a user and the page she is visiting, what is the probability that she will click on a given ad?

## About the data

__Source:__

The data we use in this project was made available by Criteo Labs as part of a Kaggle competition. They shared a week’s worth of data to develop models predicting ad click-through rate (CTR). The goal of the challenge was to benchmark the most accurate ML algorithms for CTR estimation.

__File descriptions:__

* __train.csv__ - The training set consists of a portion of Criteo's traffic over a period of 7 days. Each row corresponds to a display ad served by Criteo. Positive (clicked) and negatives (non-clicked) examples have both been subsampled at different rates in order to reduce the dataset size. The examples are chronologically ordered.
* __test.csv__ - The test set is computed in the same way as the training set but for events on the day following the training period.
* __random_submission.csv__ - A sample submission file in the correct format.

__Data fields:__

* __Label__ - Target variable that indicates if an ad was clicked (1) or not (0).
* __I1-I13__ - A total of 13 columns of integer features (mostly count features).
* __C1-C26__ - A total of 26 columns of categorical features. The values of these features have been hashed onto 32 bits for anonymization purposes. The semantic of the features is undisclosed.

In [56]:
# !wget https://www.kaggle.com/c/3934/download-all

In [44]:
TOY_EXAMPLE = PWD + '/data/toy_example.txt'
TEST_DATA = PWD + '/data/test.txt'
TRAIN_DATA = PWD + '/data/train.txt'

As we develop our code we will be using a toy example to test out our algorithm before running it on the larger dataset. We create this toy example from the large training data below.

In [45]:
!head -n 50 {TRAIN_DATA} > {TOY_EXAMPLE}

In [46]:
# Look at the first 5 lines of toy example
!head -n 5 {TOY_EXAMPLE}

0	1	1	5	0	1382	4	15	2	181	1	2		2	68fd1e64	80e26c9b	fb936136	7b4723c4	25c83c98	7e0ccccf	de7995b8	1f89b562	a73ee510	a8cd5504	b2cb9c98	37c9c164	2824a5f6	1adce6ef	8ba8b39a	891b62e7	e5ba7672	f54016b9	21ddcdc9	b1252a9d	07b5194c		3a171ecb	c5c50484	e8b83407	9727dd16
0	2	0	44	1	102	8	2	2	4	1	1		4	68fd1e64	f0cf0024	6f67f7e5	41274cd7	25c83c98	fe6b92e5	922afcc0	0b153874	a73ee510	2b53e5fb	4f1b46f3	623049e6	d7020589	b28479f6	e6c5b5cd	c92f3b61	07c540c4	b04e4670	21ddcdc9	5840adea	60f6221e		3a171ecb	43f13e8b	e8b83407	731c3655
0	2	0	1	14	767	89	4	2	245	1	3	3	45	287e684f	0a519c5c	02cf9876	c18be181	25c83c98	7e0ccccf	c78204a1	0b153874	a73ee510	3b08e48b	5f5e6091	8fe001f4	aa655a2f	07d13a8f	6dc710ed	36103458	8efede7f	3412118d			e587c466	ad3062eb	3a171ecb	3b183c5c		
0		893			4392		0	0	0		0			68fd1e64	2c16a946	a9a87e68	2e17d6f6	25c83c98	fe6b92e5	2e8a689b	0b153874	a73ee510	efea433b	e51ddf94	a30567ca	3516f6e6	07d13a8f	18231224	52b8680f	1e88c74f	74ef3502			6b3a5ca6		3a171ecb	9117a34a		
0	3	-1		0	2	0	3	0	0	1	1		0	8

In [47]:
# see how many messages/lines are in the file 
!wc -l {TOY_EXAMPLE}

50 /media/notebooks/w261_final_project/data/toy_example.txt


In [48]:
# Spark RDDs for each dataset
testRDD = sc.textFile(TOY_EXAMPLE) 
dataRDD = sc.textFile(TRAIN_DATA)

# Section 3: EDA

The first thing we want to check is the count of the ads that were clicked (label=1) and count of ads that were not clicked (label=0) and also get what percent of ads in the dataset were actually clicked. In this case we can do that by just taking the mean of the label column. 

In [51]:
def EDA1(rdd):
    total, total_clicked = None, None
    
    result = rdd.map(lambda line: line.split('\t')[0]).cache()

    total = result.count()
    total_clicked = result.reduce(lambda a, b: int(a) + int(b))
    
    return total, total_clicked

In [52]:
# test run on the toy example
start = time.time()
total, total_clicked = EDA1(testRDD)
print("Wall time: {} seconds".format(time.time() - start))

Wall time: 0.3026742935180664 seconds


In [55]:
# EDA1 on the whole train data
start = time.time()
total, total_clicked = EDA1(dataRDD)
print("Wall time: {} seconds".format(time.time() - start))

Wall time: 169.14110207557678 seconds


In [57]:
total_unclicked = total - total_clicked
mean = total_clicked / float(total)

In [58]:
print("Training dataset size: ", total)
print("Total examples with positive Label: ", total_clicked)
print("Total negative examples: ", total_unclicked)
print("Percent of ads in the data that were actually clicked: ", mean)

Training dataset size:  45840617
Total examples with positive Label:  11745438
Total negative examples:  34095179
Percent of ads in the data that were actually clicked:  0.2562233837297609


We see that 25.62% of the ads were clicked or the CTR is 25.62%. 

# Section 4: Algorithm 
    
##Algorithm description with math
##Toy example

# Section4: Implementation 